In [3]:
import numpy as np

In [7]:
?np.maximum

np.maximum 是 element-wise 取最大，同时也有broadcasting 机制

In [6]:
np.maximum([1,2,5],[2,3,4])

array([2, 3, 5])

比较下面会发现，列表与numpy 数组是具有一样的效果的

In [9]:
np.maximum(np.eye(2), [0.5,2])

array([[1. , 2. ],
       [0.5, 2. ]])

In [10]:
np.maximum(np.eye(2), np.array([0.5,2]))

array([[1. , 2. ],
       [0.5, 2. ]])

In [11]:
np.maximum(np.eye(2), np.array([[0.5,2]]))

array([[1. , 2. ],
       [0.5, 2. ]])

In [12]:
np.maximum(0, np.array([[1,-1],[-1,2]]))

array([[1, 0],
       [0, 2]])

In [13]:
np.array([[1,2],[3,4]]) + 1

array([[2, 3],
       [4, 5]])

In [14]:
np.array([[1,2],[3,4]]) + np.array([1,2])

array([[2, 4],
       [4, 6]])

In [15]:
a = np.array([[1,2],[3,4]])
a.reshape(-1,1)

array([[1],
       [2],
       [3],
       [4]])

### np.nditer

#### 简单的数组迭代

我们可以用nditer 来访问数组的每个元素，每个元素都被提供一个标准python 迭代器接口

In [19]:
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [1]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)

In [2]:
for i in odd():
    print(i)

step 1
1
step 2
3
step 3
5


In [21]:
for x in np.nditer(a):
    print(x, end = ' ')

0 1 2 3 4 5 

In [26]:
b = np.array([[3,2,4],[1,6,7],[9,8,5]])

In [27]:
for x in np.nditer(b):
    print(x, end = ' ')

3 2 4 1 6 7 9 8 5 

比较下列异同

In [25]:
a.T

array([[0, 3],
       [1, 4],
       [2, 5]])

In [22]:
for x in np.nditer(a.T):
    print(x, end = ' ')

0 1 2 3 4 5 

In [24]:
for x in np.nditer(a.T.copy(order='C')):
    print(x, end = ' ')

0 3 1 4 2 5 

In [28]:
b.T

array([[3, 1, 9],
       [2, 6, 8],
       [4, 7, 5]])

In [33]:
for x in np.nditer(b.copy(order='C')):
    print(x, end = ' ')

3 2 4 1 6 7 9 8 5 

In [35]:
for x in np.nditer(b.T):
    print(x, end = ' ')

3 2 4 1 6 7 9 8 5 

In [36]:
for x in np.nditer(b.T.copy()):
    print(x, end = ' ')

3 1 9 2 6 8 4 7 5 

In [31]:
for x in np.nditer(b.T.copy(order='C')):
    print(x, end = ' ')

3 1 9 2 6 8 4 7 5 

In [32]:
for x in np.nditer(b.T.copy(order='F')):
    print(x, end = ' ')

3 2 4 1 6 7 9 8 5 

由上面可知选择顺序与内存布局相合，order 为 'C' 时，选择顺序为按行， order 为 'F' 时选择顺序为列;copy 过后就是新的列表，有新的布局，而a 和 a.T 仍然是同一个布局。

#### 修改数组值

默认 nditer 是只读的，若要写，需要更改一些参数，然后nditer 会返回可写的缓冲数组

In [45]:
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [53]:
for x in np.nditer(a, op_flags=['readwrite']):
    x[...] = 2 * x

In [49]:
a

array([[ 0,  2,  4],
       [ 6,  8, 10]])

#### 追踪索引

在 nditer 中增加 flags ('f_index' 或 'multi_index') ,那么当前的 iterator 中的值可以用索引访问，被追踪的索引在方法 index 或者 multi_index 中

In [58]:
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [57]:
it = np.nditer(a, flags=['f_index'])
while not it.finished:
    print('%d <%s>' %(it[0], it.index))
    it.iternext()

0 <0>
1 <2>
2 <4>
3 <1>
4 <3>
5 <5>


In [60]:
it = np.nditer(a, flags=['multi_index'])
while not it.finished:
    print('%d <%s>' %(it[0], it.multi_index))
    it.iternext()

0 <(0, 0)>
1 <(0, 1)>
2 <(0, 2)>
3 <(1, 0)>
4 <(1, 1)>
5 <(1, 2)>


In [64]:
it = np.nditer(a, flags=['multi_index'], op_flags=['readwrite'])
while not it.finished:
    it[0] = it.multi_index[1] - it.multi_index[0]
    it.iternext()
a

array([[ 0,  1,  2],
       [-1,  0,  1]])

对比

In [10]:
a = np.arange(6).reshape(2,3)
it = np.nditer(a, flags=['multi_index'], op_flags=['readwrite'])
while not it.finished:
    it[0] *= 2
    it.iternext()
    print(type(it))
a

<class 'numpy.nditer'>
<class 'numpy.nditer'>
<class 'numpy.nditer'>
<class 'numpy.nditer'>
<class 'numpy.nditer'>
<class 'numpy.nditer'>


array([[ 0,  2,  4],
       [ 6,  8, 10]])

In [15]:
a = np.arange(6).reshape(2,3)
for x in np.nditer(a, flags=['multi_index'],op_flags=['readwrite']):
    x[...] = 2 * x
    print(type(x))
a

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


array([[ 0,  2,  4],
       [ 6,  8, 10]])

In [12]:
type(a)

numpy.ndarray

#### 特定数据类型

我们有时会想更改我们原本数据的类型，通常来说我们使用copy 或者 buffer 的模式，直接对原始数据操作会报错，而更好的是 buffer 模式，这种模式内存使用量较小

In [70]:
a = np.arange(6).reshape(2,3) - 3
a

array([[-3, -2, -1],
       [ 0,  1,  2]])

In [73]:
for x in np.nditer(a, op_flags=['readonly', 'copy'], op_dtypes=['complex128']):
    print(np.sqrt(x))

1.7320508075688772j
1.4142135623730951j
1j
0j
(1+0j)
(1.4142135623730951+0j)


In [74]:
for x in np.nditer(a, flags=['buffered'], op_dtypes=['complex128']):
    print(np.sqrt(x))

1.7320508075688772j
1.4142135623730951j
1j
0j
(1+0j)
(1.4142135623730951+0j)


#### 广播

np.nditer() 具有广播机制

In [80]:
a = np.arange(3)
b = np.arange(6).reshape(2,3)
for x,y in np.nditer([a,b]):
    print('%d:%d' % (x,y))

0:0
1:1
2:2
0:3
1:4
2:5


In [81]:
a = np.arange(2)
b = np.arange(6).reshape(2,3)
for x,y in np.nditer([a,b]):
    print('%d:%d' % (x,y))

ValueError: operands could not be broadcast together with shapes (2,) (2,3) 

#### 迭代器分配的输出数组

在 numpy 数组中，常见基于输入的广播的输出分配

In [83]:
def square(a):
    it = np.nditer([a, None])
    for x,y in it:
        y[...] = x*x
    return it.operands[1]
square([1,2,3])

array([1, 4, 9])

In [84]:
def square(a):
    it = np.nditer([a, None])
    for x, y in it:
        y[...] = x*x
    return it.operands[0]

square([1,2,3])

array([1, 2, 3])

当operands 传入的为 None 默认 flags 为 'allocate' 和 'writeonly',当增加一个 out 参数，默认flags 为 'readonly',所以如果我们相对它进行操作，需要显式指定一些参数

In [85]:
def square(a, out=None):
    it = np.nditer([a, out], flags=['external_loop', 'buffered'], op_flags=[['readonly'],['writeonly'
                                                                                         ,'allocate','no_broadcast']] )
    for x, y in it:
        y[...] = x*x
    return it.operands[1]

In [86]:
square([1,2,3])

array([1, 4, 9])

In [92]:
b = np.zeros((3,))
square([1,2,3], out=b)

array([1., 4., 9.])

In [93]:
b

array([1., 4., 9.])